In [ ]:
!pip install langgraph langchain langchain_groq langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.5/131.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.1/415.1 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.5 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.40
    Uninstalling langchain-core-0.3.40:
      Successfully uninstalled langchain-core-0.3.40
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.19
    Uninstalling langchain-0.3.19:
      Successfully uninstalled langchain-0.3.19


In [ ]:
!pip install arxiv wikipedia

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 5.9 MB/s eta 0:00:00
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=37d9d87c3749f5e69f68987e1970c793eb044c9d20053d19c4d9411591f4a1a3
  Stored in directory: /root/.cache/pip/wheels/8f/ab/cb/45ccc40522d3a1c41e1d2ad53b8f33a62f394011ec38cd71c6
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6047 sha256=25365202d194a06534c1d0656c04c1627cd0eaa4ade1f1b0b81898a6adc63583
  Stored in directory: /root/.cache/pip/wheels/3b/25/2a/105d6a15df6914f4d15047691c6c28f9052cc1173e40285d03
Successfully built wikipedia sgmllib3k


In [ ]:
from typing  import Annotated
from typing_extensions import TypedDict

In [ ]:
# working with tools
from langchain_community.utilities import ArxivAPIWrapper,WikipediaAPIWrapper
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
arxiv_wrapper= ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=300)
arxiv_tool=ArxivQueryRun(api_wrapper=arxiv_wrapper)
Wikipedia_wrapper = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=300)
Wikipedia_tool=WikipediaQueryRun(api_wrapper=Wikipedia_wrapper)


In [ ]:
Wikipedia_tool.invoke("what is gen Ai?")

In [ ]:
arxiv_tool.invoke("content Aggregator")

In [ ]:
tools= [Wikipedia_tool]

In [ ]:
from langgraph.graph.message import add_messages
class State(TypedDict):
    messages:Annotated[list,add_messages]

In [ ]:
from langgraph.graph import StateGraph, START,END

In [ ]:
graph_builder = StateGraph(State)

In [ ]:
from langchain_groq import ChatGroq

In [ ]:
from google.colab import userdata
groq_api_key = userdata.get('groq_api_key')

In [ ]:
llm = ChatGroq(groq_api_key =groq_api_key, model_name = "gemma2-9b-it")
llm

In [ ]:
llm_with_tools = llm.bind(tools=tools)

In [ ]:
def chatbot(state :State):
  return {"messages":[llm_with_tools.invoke(state["messages"])]}

In [ ]:
from langgraph.prebuilt import ToolNode,tools_condition

In [ ]:
graph_builder.add_node("chatbot",chatbot)
graph_builder.add_edge(START,"chatbot")
tool_node =  ToolNode(tools=tools)
graph_builder.add_node("tools",tool_node)
graph_builder.add_conditional_edges("chatbot",tools_condition)
graph_builder.add_edge("tools","chatbot")
graph_builder.add_edge("chatbot",END)

In [ ]:
graph = graph_builder.compile()

In [ ]:
from IPython.display import Image,display
try :
  display (Image(graph.get_graph().draw_mermaid_png()))
except Exception:
  pass

In [ ]:
from langgraph.prebuilt import ToolNode, tools_condition

In [ ]:
user_input = "Hi there!, My name is "
user_input = {"messages": [{"role": "user", "content": user_input}]}
events = graph.stream(user_input, stream_mode="messages")
for event in events:
    event["messages"][-1].pretty_print()

# events = graph.stream(
#     {"messages": [("user", user_input)]}, stream_mode="values")
# for event in events:
#     event["messages"][-1].pretty_print()


In [ ]:
!pip show langgraph

In [ ]:
for event in events:
  print(type(event))

In [ ]:

for msg,metadata in  graph.stream({"topic":"icecream"}, stream_mode="messages"
):
  print(type(metadata))
  print(metadata)


In [ ]:

# Install necessary packages
!pip install langgraph langchain langchain_groq langchain_community
!pip install arxiv wikipedia

# Import necessary modules
from typing import Annotated
from typing_extensions import TypedDict

# Working with tools
from langchain_community.utilities import ArxivAPIWrapper, WikipediaAPIWrapper
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun

# Initialize Arxiv API Wrapper
arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=300)
arxiv_tool = ArxivQueryRun(api_wrapper=arxiv_wrapper)

# Initialize Wikipedia API Wrapper
wikipedia_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=300)
wikipedia_tool = WikipediaQueryRun(api_wrapper=wikipedia_wrapper)

# Invoke Wikipedia tool
wikipedia_tool.invoke("what is gen Ai?")

# Define Memory class
class Memory(TypedDict):
    chat_history: Annotated[list[str], "chat_history"]

# Define ResearchAgent class
class ResearchAgent:
    def __init__(self):
        self.arxiv_tool = ArxivQueryRun(api_wrapper=ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=1000))
        self.wikipedia_tool = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=500))

    def search_arxiv(self, query: str):
        return self.arxiv_tool.invoke(query)

    def search_wikipedia(self, query: str):
        return self.wikipedia_tool.invoke(query)



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.5/131.5 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.1/415.1 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.40
    Uninstalling langchain-core-0.3.40:
      Successfully uninstalled langchain-core-0.3.40
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.19
    Uninstalling langchain-0.3.19:
      Successfully uninstalled langchain-0.3.19
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━

In [ ]:
agent = ResearchAgent()
print(agent.search_arxiv("news aggrigator"))

Published: 2022-03-23
Title: MM-Rec: Multimodal News Recommendation
Authors: Chuhan Wu, Fangzhao Wu, Tao Qi, Yongfeng Huang
Summary: Accurate news representation is critical for news recommendation. Most of
existing news representation methods learn news representations only from news
texts while ignore the visual information in news like images. In fact, users
may click news not only because of the interest in news titles but also due to
the attraction of news images. Thus, images are useful for representing news
and predicting user behaviors. In this paper, we propose a multimodal news
recommendation method, which can incorporate both textual and visual
information of news to learn multimodal news representations. We first extract
region-of-interests (ROIs) from news images via object detection. Then we use a
pre-trained visiolinguistic model to encode both news texts and news image ROIs
and model their inherent relatedness using co-attentional Transformers. In
addition, we propose a

In [ ]:
print(agent.search_wikipedia("Artificial Intelligence"))

Page: Artificial intelligence
Summary: Artificial intelligence (AI) refers to the capability of computational systems to perform tasks typically associated with human intelligence, such as learning, reasoning, problem-solving, perception, and decision-making. It is a field of research in computer science that develops and studies methods and software that enable machines to perceive their environment and use learning and intelligence to take actions that maximize their chances of achieving defin


In [ ]:
# Define function to update memory
def update_memory(memory: Memory, new_entry: str) -> Memory:
    memory["chat_history"].append(new_entry)
    return memory

In [ ]:
# Example memory usage
memory = Memory(chat_history=[])
memory = update_memory(memory, "User asked about AI.")
print(memory)



{'chat_history': ['User asked about AI.']}
